In [1]:
%env TOKENIZERS_PARALLELISM=false
%env WANDB_DISABLED=true
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AdamW, 
    AutoConfig, 
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    EarlyStoppingCallback
)

from datasets import Dataset, load_metric

import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

env: TOKENIZERS_PARALLELISM=false
env: WANDB_DISABLED=true


In [2]:
training_headline_df = pd.read_csv('../input/reddit-fin-headlines-data/2k_new_labelled.csv')[['sentiment', 'title']]
serving_headline_df = pd.read_csv('../input/reddit-fin-headlines-data/8k-labelled-vader.csv')[['title']]

kaggle_headline_df = pd.read_csv('../input/sentiment-analysis-for-financial-news/all-data.csv', sep=',', encoding='latin-1',names = ["sentiment","title"])
combined_df = pd.concat([training_headline_df, kaggle_headline_df]).sample(frac=1)

# Dataset

In [3]:
def preprocess_data(headline_df, tokenizer):
    def tokenize_fn(examples):
        return tokenizer(examples['text'], truncation=True)
    
    headline_df['sentiment'] = LabelEncoder().fit_transform(headline_df['sentiment'])
    X, y = headline_df['title'].values, headline_df['sentiment'].values
    # train : val : test = 0.8 : 0.1 : 0.1
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.1, stratify=y)
    xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.1, stratify=ytrain)
    
    train_dataset_raw = Dataset.from_dict({'text':xtrain, 'labels':ytrain})
    valid_dataset_raw = Dataset.from_dict({'text':xval, 'labels': yval})
    test_dataset_raw = Dataset.from_dict({'text':xtest, 'labels': ytest})

    train_dataset = train_dataset_raw.map(tokenize_fn, batched=True)
    valid_dataset = valid_dataset_raw.map(tokenize_fn, batched=True)
    test_dataset = test_dataset_raw.map(tokenize_fn, batched=True)
    
    return train_dataset, valid_dataset, test_dataset

In [4]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall}

In [5]:
def create_trainer(model, tokenizer, train_dataset, valid_dataset):
    data_collator = DataCollatorWithPadding(tokenizer)
    
    train_args = TrainingArguments(
        './Finbert Trained/',
        per_device_train_batch_size=16,
        per_device_eval_batch_size=2*16,
        num_train_epochs=50,
        learning_rate=2e-5,
        weight_decay=0.01,
        warmup_ratio=0.1,    
        do_eval=True,
        do_train=True,
        do_predict=True,
        evaluation_strategy='epoch',
        load_best_model_at_end = True,
        metric_for_best_model='f1',
        save_strategy="epoch",
        save_total_limit = 2
    )

    trainer = Trainer(
        model,
        train_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(5)]
    )

    return trainer

In [6]:
def get_test_metric(trainer, valid_dataset):
    output = trainer.predict(
        test_dataset=valid_dataset,
        )
    return compute_metrics(output)

In [7]:
model_data = {
    "train_on_kg": kaggle_headline_df,
    "train_on_reddit": training_headline_df,
    "transfer_learning_on_reddit": training_headline_df,
    "combined_data": combined_df 
}

val_dfs = {}

for model_name in model_data:
    print("---------------------------------------------------")
    print(f"Start training for {model_name}")
    df = model_data[model_name]
    
    if model_name == "transfer_learning_on_reddit":
        pretrained = 'finbert_finetuned_train_on_kg.bin'
        
    else:
        pretrained = 'ProsusAI/finbert'
    model = AutoModelForSequenceClassification.from_pretrained(pretrained)
    tokenizer = AutoTokenizer.from_pretrained(pretrained)
    
    train_dataset, valid_dataset, test_dataset = preprocess_data(df, tokenizer)
    
    trainer = create_trainer(model, tokenizer, train_dataset, valid_dataset)
    trainer.train()
    
    trainer.save_model(f'finbert_finetuned_{model_name}.bin')
    
    print("---------------------------------------------------")
    print("Test Metric")
    trainer.evaluate()
    test_metric = get_test_metric(trainer, test_dataset)
    print(test_metric)
    print(f"Finish {model_name}")
    print("---------------------------------------------------")

---------------------------------------------------
Start training for train_on_kg


Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.698301,0.714976,0.602082,0.675391,0.580817
2,1.418800,0.389050,0.855072,0.836472,0.854693,0.829547
3,0.348800,0.408046,0.845411,0.834123,0.823168,0.848047
4,0.348800,0.488727,0.851852,0.834572,0.845363,0.829117
5,0.197000,0.608906,0.847021,0.833166,0.834663,0.831785
6,0.126700,0.679744,0.847021,0.825773,0.841353,0.812887
7,0.126700,0.833472,0.827697,0.809765,0.811847,0.808020


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 621
  Batch size = 32
Saving model checkpoint to ./Finbert Trained/checkpoint-349
Configuration saved in ./Finbert Trained/checkpoint-349/config.json
Model weights saved in ./Finbert Trained/checkpoint-349/pytorch_model.bin
tokenizer config file saved in ./Finbert Trained/checkpoint-349/tokenizer_config.json
Special tokens file saved in ./Finbert Trained/checkpoint-349/special_tokens_map.json
Deleting older checkpoint [Finbert Trained/checkpoint-624] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 621
  Batch size = 32
Saving model checkpoint to ./Finbert Trained/checkpoint-698
Configuration saved in ./Finbert

---------------------------------------------------
Test Metric


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 690
  Batch size = 32


{'accuracy': 0.8652173913043478, 'f1': 0.8512346671553813, 'precision': 0.8524576884822576, 'recall': 0.8563947748158274}
Finish combined_data
---------------------------------------------------


In [8]:
id2label = {
    0: "positive",
    1: "negative",
    2: "neutral"
  }

# Test

In [9]:
def prep_serving_data(headline_df, tokenizer):

    def tokenize_fn(examples):
        return tokenizer(examples['text'], truncation=True)

    X = headline_df['title'].values
    
    test_dataset_raw = Dataset.from_dict({'text':X})

    test_dataset = test_dataset_raw.map(tokenize_fn, batched=True)
    
    return test_dataset

In [10]:
def get_pred(dataset, tokenizer):
    data_collator = DataCollatorWithPadding(tokenizer)
    
    # arguments for Trainer
    test_args = TrainingArguments(
        output_dir = pretrained,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = 64,   
        dataloader_drop_last = False    
    )

    # init trainer
    trainer = Trainer(
                  model = model, 
                  args = test_args, 
                  data_collator=data_collator,
                  compute_metrics = compute_metrics)
    
    test_results = trainer.predict(dataset)
    
    return test_results

In [11]:
metric_dict = {}
df_dict = {}

In [12]:
model_cpt = {
    "train_on_kg": './finbert_finetuned_train_on_kg.bin',
    "train_on_reddit": './finbert_finetuned_train_on_reddit.bin',
    "transfer_learning_on_reddit": './finbert_finetuned_transfer_learning_on_reddit.bin',
    "combined_data": './finbert_finetuned_combined_data.bin' 
}

In [13]:
training_headline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 0 to 2046
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  2047 non-null   int64 
 1   title      2047 non-null   object
dtypes: int64(1), object(1)
memory usage: 32.1+ KB


In [14]:
for model_name in model_cpt:
    print("---------------------------------------------------")
    print(f"Start serving for {model_name} using serving_headline_reddit")
    pretrained = model_cpt[model_name]
    
    model = AutoModelForSequenceClassification.from_pretrained(pretrained)
    tokenizer = AutoTokenizer.from_pretrained(pretrained)
    
    test_dataset = prep_serving_data(serving_headline_df, tokenizer)
    test_results = get_pred(test_dataset, tokenizer)
    preds = test_results.predictions.argmax(-1)

    print("---------------------------------------------------")
    
    submission = pd.DataFrame()
    submission['title'] = test_dataset['text']
    submission['prediction'] = preds
    submission['prediction'] = submission['prediction'].apply(lambda x:id2label[x])
    submission.to_csv(f'finbert_{model_name}_serving.csv', index=False)
    df_dict[f'finbert_{model_name}'] = submission

    
    print(f"Finish {model_name}")
    print("---------------------------------------------------")

loading configuration file ./finbert_finetuned_train_on_kg.bin/config.json
Model config BertConfig {
  "_name_or_path": "./finbert_finetuned_train_on_kg.bin",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522


---------------------------------------------------
Start serving for train_on_kg using serving_headline_reddit


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ./finbert_finetuned_train_on_kg.bin.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
Didn't find file ./finbert_finetuned_train_on_kg.bin/added_tokens.json. We won't load it.
loading file ./finbert_finetuned_train_on_kg.bin/vocab.txt
loading file ./finbert_finetuned_train_on_kg.bin/tokenizer.json
loading file None
loading file ./finbert_finetuned_train_on_kg.bin/special_tokens_map.json
loading file ./finbert_finetuned_train_on_kg.bin/tokenizer_config.json


  0%|          | 0/9 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 8409
  Batch size = 64


...

---------------------------------------------------
Finish transfer_learning_on_reddit
---------------------------------------------------
---------------------------------------------------
Start serving for combined_data using serving_headline_reddit


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ./finbert_finetuned_combined_data.bin.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
Didn't find file ./finbert_finetuned_combined_data.bin/added_tokens.json. We won't load it.
loading file ./finbert_finetuned_combined_data.bin/vocab.txt
loading file ./finbert_finetuned_combined_data.bin/tokenizer.json
loading file None
loading file ./finbert_finetuned_combined_data.bin/special_tokens_map.json
loading file ./finbert_finetuned_combined_data.bin/tokenizer_config.json


  0%|          | 0/9 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 8409
  Batch size = 64


---------------------------------------------------
Finish combined_data
---------------------------------------------------


In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until
